### Part A. The way I downloaded the dataset

In [17]:
import pandas as pd,numpy as np,os,glob
import spacy
df=pd.read_csv('Historic dataset on 2020 11 22_v7_Compleat.csv')

2022-12-05 17:11:09.269073: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-05 17:11:09.269090: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/sourin/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (10,29,31,32,35,36,40,44,45,46,48,51,54,65,84,89,90,93,98,99,100,105,108,115,118,122,123,125,130,134,138,142,143,144,146,149,152,163,166,167,171,172,173,175,178,181,192,194,225,271,272,273,277,315,335,336,342,348,371,373,374,376,380,384,387) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
for x1,x2 in enumerate(df['Ref_DOI_number'].unique()):
    if x1==200:print(x2) #prints the doi of which I search the abstract

10.3390/nano9091220


In [ ]:
## the above printed doi is searched and the corresponding abstract is copy and pasted as in the following cell.

In [12]:
# f contains all the 200 downloaded abstracts
f=pd.read_csv('project_abs.csv')


f.loc[200,'text']='The mesoporous TiO2 nanoparticle-based scaffold structure is the best electron transport layer (ETL) for perovskite solar cells (PSCs) and is still used in most PSCs with optimal photovoltaic characteristics. However, the high sintering temperature of TiO2 nanoparticles required to remove binders from the TiO2 paste limits PSC application to flexible electronics. In this study, a simple interface modification process involving ethanol rinsing is developed to enhance the photovoltaic characteristics of low-temperature processed PSCs. This easy and fast technique could enable remarkable performance by PSCs by significantly increasing the fill factor and current density, leading to a power conversion efficiency more than four times that of untreated solar cells'
print(f.tail(5))
#f.to_csv('project_abs.csv',index=False)

In [18]:
d2=pd.DataFrame(columns=['text'])

def find(sen,id1):
    
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(sen)
    sen=list(doc.sents)
    names=['added','fabricate','synthesized','prepared','produced','process','synthesize','improve','manufactured']
    names2=[ps.stem(i) for i in names]
    rcv=[]
#     print(names2)
    for x1,x2 in enumerate(sen):
        x2=str(x2).split(' ')
        for x3,x4 in enumerate(x2):
#             print(ps.stem(x4))
            if ps.stem(x4) in names2:
#                 print(x2,' id:',id1)
                rcv.append(x2)
                break
    return rcv

store=[]
for id1,x in enumerate(f['text']):
    t=find(x,id1)
    if len(t)!=0:
        if len(t)>1:
            for x5 in t:
                store.append(x5)
        else:
            store.append(t[0])
ctr=0
for p1,p2 in enumerate(store):
    p2.append(' ')
    for p3 in p2:
        d2.loc[ctr,'text']=p3
        ctr+=1
d2.head()

,text
0,In
1,the
2,present
3,"study,"
4,Er
...,...
9427,of
9428,low-temperature
9429,processed
9430,PSCs.


### Part B. the downloaded 200 abstracts which I saved in the train.csv file

In [8]:
df=pd.read_csv('train.csv')
df.shape

(4914, 2)

Labeling the words synonymous to "grown" and stemming them in the dataset. Following is the list.

### 'added','fabricate','synthesized','prepared','produced','process','synthesize','improve','manufactured' 



In [9]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()

names=['added','fabricate','synthesized','prepared','produced','process','synthesize','improve','manufactured']
names2=[ps.stem(i) for i in names]

print(names2)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


['ad', 'fabric', 'synthes', 'prepar', 'produc', 'process', 'synthes', 'improv', 'manufactur']


## Part C. Labelling the words in the dataset

### chemical elements as B
### members of names2 as I
### other words as O

In [19]:

label_p=[i1 for i1,i2 in enumerate(d2['text']) if '3' in i2 and len(i2)>=4 or 'pero' in i2 or 'PSC' in i2]
len(label_p)

label_m=[i1 for i1,i2 in enumerate(d2['text']) if ps.stem(i2) in names2]

for a1,a2 in enumerate(d2['text']):
    if a1 in label_p:
        d2.loc[a1,'label']='B'
    elif a1 in label_m:
        d2.loc[a1,'label']='I'
    else: d2.loc[a1,'label']='O'
        


In [20]:
d2

,text,label
0,In,O
1,the,O
2,present,O
3,"study,",O
4,Er,O
...,...,...
9427,of,O
9428,low-temperature,O
9429,processed,I
9430,PSCs.,B


## Part D. Now merge them with BC4CHEMD dataset

In [21]:
import csv
df_p = pd.read_csv('parent_train.tsv', header = None, delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')

df_p.shape

(893685, 2)

#### stem the words present in this big data only for label I

In [ ]:
for x1,x2 in enumerate(df[df.columns[0]]):
    try:
        if ps.stem(x2) in names2:
            k+=1
            print(x2,ps.stem(x2))
            df_p[df_p.columns[1]][x1]='I'
    except:
        print("this made issue {}".format(x2))

In [22]:
x5=df_p[df_p.columns[0]].to_numpy()
x6=df_p[df_p.columns[1]].to_numpy()

### merge d2 with parent df_p

In [23]:


df3=pd.DataFrame(columns=['text','label'])
df3['text']=x5
df3['label']=x6
df4=d2.append(df3)
df4=df4.reset_index(drop=True)

df4.head(10)

,text,label
0,In,O
1,the,O
2,present,O
3,"study,",O
4,Er,O
5,and,O
6,Yb,O
7,co-doped,O
8,TiO2,O
9,(Er-Yb:TiO2),O


In [26]:
df4.shape

(903117, 2)

In [27]:
df4.to_csv('df4.csv',index=False)

### save it as tsv because bio-bert format requires it

In [28]:
with open('df4.csv','r') as csvin, open('train.tsv', 'w') as tsvout:
    csvin = csv.reader(csvin)
    tsvout = csv.writer(tsvout, delimiter='\t')

    for row in csvin:
        tsvout.writerow(row)